# train_models

Train the face detection and landmark localisation models for Zeno's face based on our annotation.

## Step 1: Import models

In [5]:
import os
import cv2
import glob
import dlib
import time
import numpy as np
import pandas as pd
try:
    from ConfigParser import ConfigParser    # If using Python 2.7
except ImportError:
    from configparser import ConfigParser    # If using Python 3.5
config = ConfigParser()
config.read('config.ini')
print('All modules imported.')

All modules imported.


## Step 2: Prepare the data structure for training

In [7]:
# Load images and annotation
annotations = pd.read_pickle(os.path.realpath(os.path.join('./dataset', 'annotations.pkl')))
face_detection_scale = config.getfloat('facial_landmark_tracker', 'face_detection_scale')
face_detection_images = []
face_detection_groundtruth = []
shape_predictor_images = []
shape_predictor_groundtruth = []
last_check_time = time.time()
for idx in range(annotations.shape[0]):
    entry = annotations.iloc[idx]
    image_path = os.path.realpath(os.path.join('./dataset', entry['session'], 
                                               '%06d.png' % entry['index']))
    if type(entry['face_box']) != type(np.nan) or not np.isnan(entry['face_box']):
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY)
        image_size = (image.shape[1], image.shape[0])
        face_detection_size = (max(round(image_size[0] * face_detection_scale), 1), 
                               max(round(image_size[1] * face_detection_scale), 1))
        if face_detection_size != image_size:
            face_detection_images.append(cv2.resize(image, face_detection_size))
        else:
            face_detection_images.append(face_detection_image)
        face_box = [int(round(x * face_detection_scale)) for x in list(entry['face_box'])]
        face_detection_groundtruth.append([dlib.rectangle(face_box[0], face_box[1], 
                                                          face_box[0] + face_box[2] - 1, 
                                                          face_box[1] + face_box[3] - 1)])
        if type(entry['landmarks']) != type(np.nan) or not np.isnan(entry['landmarks']):
            shape_predictor_images.append(image)
            bounding_box = dlib.rectangle(entry['face_box'][0], entry['face_box'][1], 
                                          entry['face_box'][0] + entry['face_box'][2] - 1, 
                                          entry['face_box'][1] + entry['face_box'][3] - 1)
            landmarks = [dlib.point(int(round(pts[0])), int(round(pts[1]))) 
                         for pts in entry['landmarks']]
            shape_predictor_groundtruth.append([dlib.full_object_detection(bounding_box, 
                                                                           landmarks)])
    current_time = time.time()
    if last_check_time < current_time - 10.0:
        last_check_time = current_time
        print('%d face detection samples and %d landmark localisation have been prepared.' % 
              (len(face_detection_groundtruth), len(shape_predictor_groundtruth)))
print('All %d face detection samples and %d landmark localisation have been prepared.' % 
      (len(face_detection_groundtruth), len(shape_predictor_groundtruth)))

468 face detection samples and 130 landmark localisation have been prepared.
955 face detection samples and 275 landmark localisation have been prepared.
All 1200 face detection samples and 349 landmark localisation have been prepared.


## Step 3: Train the face detector model

## Step 4: Train the shape predictor model

In [99]:
options = dlib.shape_predictor_training_options()
options.be_verbose = config.getboolean('shape_predictor_training_options', 
                                       'be_verbose')
options.cascade_depth = config.getint('shape_predictor_training_options', 
                                      'cascade_depth')
options.feature_pool_region_padding = config.getfloat('shape_predictor_training_options', 
                                                      'feature_pool_region_padding')
options.feature_pool_size = config.getint('shape_predictor_training_options', 
                                          'feature_pool_size')
options.lambda_param = config.getfloat('shape_predictor_training_options', 
                                       'lambda_param')
options.nu = config.getfloat('shape_predictor_training_options', 'nu')
options.num_test_splits = config.getint('shape_predictor_training_options', 
                                        'num_test_splits')
options.num_trees_per_cascade_level = config.getint('shape_predictor_training_options', 
                                                    'num_trees_per_cascade_level')
options.oversampling_amount = config.getint('shape_predictor_training_options', 
                                            'oversampling_amount')
options.tree_depth = config.getint('shape_predictor_training_options', 'tree_depth')
shape_predictor = dlib.train_shape_predictor(images, groundtruth, options)
shape_predictor_model_path = os.path.realpath(os.path.join('./models', 'zeno_face_tracker_model.dat'))
shape_predictor.save(shape_predictor_model_path)
print('Shape predictor model has been trained and saved to: ' + shape_predictor_model_path)

Shape predictor model has been trained and saved to: D:\hhj\zeno_face_tracker\models\zeno_face_tracker_model.dat


## Step 4: Test the model on live video

In [92]:
model.save('lala.dat')n

In [2]:
annotations = pd.read_pickle(os.path.realpath(os.path.join('./dataset', 'annotations.pkl')))

In [12]:
import numpy as np
np.isnan(annotations.iloc[0]['aligned_landmarks'])

True

In [11]:
np.nan == np.nan

False

In [14]:
face_detection_scale = max(config.getfloat('facial_landmark_tracker', 
                                           'face_detection_scale'), 1e-6)

In [15]:
face_detection_scale

0.5